In [149]:
import os
import glob
import pickle
import time
from math import log2
from itertools import cycle, product
from collections import defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LogNorm
%matplotlib inline

# Settings

In [150]:
# Data logs root directory
LOG_DIR = "logs_alphabeta/logistic"

# The following should be the same as the one used in run_experiment.py
DATASETS = ("a9a", "w8a", "rcv1", "real-sim",)
OPTIMIZERS = ("Adam", "SARAH", "L-SVRG")
T = 100  # Use 2xT used in run_experiment.py

# These are the metrics collected in the data logs
METRICS = ("loss", "gradnorm", "error")
METRIC = "error"  # choose metric

# These are aggregators for comparing multi-seed runs
AGGS = ("mean", "median")
AGG = "mean"  # choose aggregator

# Downsample this number of effective passes by averaging them
AVG_DOWNSAMPLE = 5

# These are the logs columns: effective passes + metrics
LOG_COLS = ["ep"] + list(METRICS)

# These are the hyperparameters of interest
ARG_COLS = ["lr", "alpha", "beta2", "precond"]

# Plots will be generated for this hyperparams/args setting.
# 'corrupt' should be the scale/suffix of the dataset as a string or 'none'.
FILTER_ARGS = {
    "corrupt": "none",
    "weight_decay": 0,
}

# Ignore all runs containing 'any' of these hyperparams.
IGNORE_ARGS = {}

### Utility functions for loading data

In [151]:
def ignore(args_dict):
    return any(args_dict[arg] in IGNORE_ARGS[arg]
               for arg in IGNORE_ARGS.keys() if arg in args_dict)


def loaddata(fname):
    with open(fname, 'rb') as f:
        data = pickle.load(f)
    return data


def contain_dict(dict1, dict2):
    return all(dict1[k] == v for k, v in dict2.items() if k in dict1)

# Gathering data and finding best hyperparameters for each (optimizer, dataset) combination

In [152]:
def unpack_args(fname):
    """
    Recover all args given file path.
    """
    args = {}
    # unpack path
    dirname, logname = os.path.split(fname)
    logdir, args["dataset"] = os.path.split(dirname)
    # parse args
    args["optimizer"], argstr = logname.split("(")
    argstr, _ = argstr.split(")")  # remove ').pkl'
    args_dict = {k:v for k,v in [s.split("=") for s in argstr.split(",")]}

    # Extract args
    if args["dataset"][-1] == ")":
        args["corrupt"] = args["dataset"][args["dataset"].index("("):]
    else:
        # It is very unlikely that the original dataset name will end with ')'
        args["corrupt"] = "none"

    if "seed" in args_dict:
        args["seed"] = int(args_dict["seed"])
    else:
        args["seed"] = 0

    args["BS"] = int(args_dict["BS"])
    args["lr"] = float(args_dict["lr"])
    if "weight_decay" in args_dict:
        args["weight_decay"] = float(args_dict["weight_decay"])
    else:
        args["weight_decay"] = 0
    if "lr_decay" in args_dict:
        args["lr_decay"] = float(args_dict["lr_decay"])
    else:
        args["lr_decay"] = 0
    if "p" in args_dict:
        args["p"] = args_dict["p"]
    if "precond" in args_dict:
        args["precond"] = args_dict["precond"]
        args["beta2"] = float(args_dict["beta2"])
        args["alpha"] = float(args_dict["alpha"])
    else:
        args["precond"] = "none"
        args["alpha"] = "none"
        args["beta2"] = "none"
    
    if args["optimizer"] == "Adam":
        args["beta2"] = float(args_dict["beta2"])
        args["alpha"] = 1e-8

    return args


def get_logs(logdir, dataset, optimizer, **filter_args):
    """
    Return all logs in 'logdir' containing the filter hyperparams.
    Dataset name should contain feature scaling, if any
    e.g. 'dataset' or 'dataset(k_min,k_max)'.
    
    Returns the data in the log file and its arguments/hyperparams.
    """
    remove_empty_data = False
    # Add
    if "corrupt" in filter_args and filter_args['corrupt'] != "none":
        # Add scale suffix to specify dataset    
        dataset += filter_args['corrupt']
    else:
        # No setting specified, use wildcard to match all suffixes
        dataset += "*"
    # Find all files matching this pattern
    for fname in glob.glob(f"{logdir}/{dataset}/{optimizer}(*).pkl"):
        exp_args = unpack_args(fname)
        # Skip if filter_args do not match args of this file
        if not contain_dict(exp_args, filter_args):
            continue
        # Load data
        data = loaddata(fname)
        # Handle empty data files
        if len(data) == 0:
            print(fname, "has no data!")
            """
            if "y" == input("Remove empty log files in the future? (y/n)"):
                remove_empty_data = True
            if remove_empty_data:
                try:
                    print("Removing", fname)
                    os.remove(fname)
                except OSError as e:
                    print ("Error: %s - %s." % (e.filename, e.strerror))
            """
            continue
        # @XXX: hack to correct wrong initial ep>0 for L-SVRG
        ep0 = data[0,0]
        if ep0 > 0.:
            data[:,0] -= ep0
        yield data, exp_args

        
# Gather data
all_dfs = {}
start_time = time.time()
for exp in product(DATASETS, OPTIMIZERS):
    exp_df = pd.DataFrame()
    # Get all log data given the experiment and filter args
    for data, args in get_logs(LOG_DIR, *exp, **FILTER_ARGS):
        if ignore(args):
            continue
        # Get experiment log data
        df = pd.DataFrame(data[:, :4], columns=LOG_COLS)
        # Get args of interest
        for col in ARG_COLS:
            df[col] = args[col]
        # Downsample by averaging metrics every AVG_DOWNSAMPLE epoch.
        df["ep"] = np.ceil(df["ep"] / AVG_DOWNSAMPLE) * AVG_DOWNSAMPLE
        df = df.groupby(["ep"] + ARG_COLS).mean().reset_index()
        # Get data up to the prespecified epoch T
        df = df[df["ep"] <= T]
        # @TODO: is this efficient?
        exp_df = exp_df.append(df, ignore_index=True)
    # Record all runs of exp in a single dataframe
    all_dfs[exp] = exp_df

    if len(exp_df) == 0:
        print("No log data found for this experiment!")
        print("- Experiment:", exp)
        print("- filter_args:", FILTER_ARGS)
        continue
data_gather_time = time.time() - start_time
print(f"Data frame lengths:")
for exp, df in all_dfs.items():
    print(f"{exp} -> {len(df)} data rows -> {len(df) // T} runs")
print(f"Took about {data_gather_time:.2f} seconds to gather all these data.")

logs_alphabeta/logistic/w8a/Adam(seed=3,BS=128,lr=0.0625,beta2=0.999).pkl has no data!
logs_alphabeta/logistic/w8a/Adam(seed=3,BS=128,lr=0.015625,beta2=0.999).pkl has no data!
logs_alphabeta/logistic/w8a/Adam(seed=3,BS=128,lr=0.25,beta2=0.999).pkl has no data!
logs_alphabeta/logistic/w8a/Adam(seed=3,BS=128,lr=4,beta2=0.999).pkl has no data!
logs_alphabeta/logistic/w8a/Adam(seed=3,BS=128,lr=0.00390625,beta2=0.999).pkl has no data!
logs_alphabeta/logistic/w8a/Adam(seed=3,BS=128,lr=1,beta2=0.999).pkl has no data!
logs_alphabeta/logistic/w8a/Adam(seed=3,BS=128,lr=16,beta2=0.999).pkl has no data!


In [163]:
for i, (exp, df) in enumerate(all_dfs.items()):
    if i == 3: break
    print(exp)
    display(df)

('a9a', 'Adam')


,ep,lr,alpha,beta2,precond,loss,gradnorm,error
0,0.0,0.0625,1.000000e-08,0.999,none,0.693147,0.032853,0.500000
1,5.0,0.0625,1.000000e-08,0.999,none,0.340162,0.000939,0.156502
2,10.0,0.0625,1.000000e-08,0.999,none,0.326467,0.000172,0.152164
3,15.0,0.0625,1.000000e-08,0.999,none,0.325896,0.000108,0.151791
4,20.0,0.0625,1.000000e-08,0.999,none,0.326283,0.000157,0.152156
...,...,...,...,...,...,...,...,...
2998,80.0,4.0000,1.000000e-08,0.999,none,0.543354,0.003034,0.186708
2999,85.0,4.0000,1.000000e-08,0.999,none,0.542057,0.003305,0.194414
3000,90.0,4.0000,1.000000e-08,0.999,none,0.591068,0.004067,0.194331
3001,95.0,4.0000,1.000000e-08,0.999,none,0.559465,0.003294,0.191398


('a9a', 'SARAH')


,ep,lr,alpha,beta2,precond,loss,gradnorm,error
0,0.0,0.0625,1.000000e-07,0.995,hutchinson,0.693147,0.032853,0.500000
1,5.0,0.0625,1.000000e-07,0.995,hutchinson,0.826648,0.000253,0.162838
2,10.0,0.0625,1.000000e-07,0.995,hutchinson,2.453245,0.000003,0.152783
3,15.0,0.0625,1.000000e-07,0.995,hutchinson,47.543183,0.000065,0.166875
4,20.0,0.0625,1.000000e-07,0.995,hutchinson,55.804485,0.000005,0.155937
...,...,...,...,...,...,...,...,...
27688,80.0,0.0625,1.000000e-07,0.995,hutchinson,41808.893660,0.001817,0.195280
27689,85.0,0.0625,1.000000e-07,0.995,hutchinson,32915.513302,0.001698,0.212634
27690,90.0,0.0625,1.000000e-07,0.995,hutchinson,36286.816012,0.002485,0.218956
27691,95.0,0.0625,1.000000e-07,0.995,hutchinson,33416.226438,0.001622,0.209002


('a9a', 'L-SVRG')


,ep,lr,alpha,beta2,precond,loss,gradnorm,error
0,0.0,16.0000,1.000000e-07,0.95,hutchinson,6.931472e-01,3.285310e-02,0.500000
1,5.0,16.0000,1.000000e-07,0.95,hutchinson,3.183355e+05,1.905281e-02,0.235070
2,10.0,16.0000,1.000000e-07,0.95,hutchinson,1.496375e+06,1.728026e-02,0.208695
3,15.0,16.0000,1.000000e-07,0.95,hutchinson,1.240142e+06,1.450116e-02,0.194107
4,20.0,16.0000,1.000000e-07,0.95,hutchinson,1.429925e+06,1.801447e-02,0.216256
...,...,...,...,...,...,...,...,...
27715,80.0,0.0625,1.000000e-01,0.99,hutchinson,3.266063e-01,1.116324e-06,0.153082
27716,85.0,0.0625,1.000000e-01,0.99,hutchinson,3.263014e-01,9.412707e-07,0.153152
27717,90.0,0.0625,1.000000e-01,0.99,hutchinson,3.261063e-01,8.351229e-07,0.153073
27718,95.0,0.0625,1.000000e-01,0.99,hutchinson,3.259120e-01,7.346767e-07,0.153203


## Get best hyperparams

In [164]:
for exp, df in all_dfs.items():
    if exp[1] == "Adam": continue
    alphas = set([] if "alpha" not in ARG_COLS else df["alpha"])
    betas = set([] if "beta2" not in ARG_COLS else df["beta2"])
    lrs = set([] if "lr" not in ARG_COLS else df["lr"])
    preconds = set([] if "precond" not in ARG_COLS else df["precond"])
    break

best_dfs = {}
best_dfs_alpha = {}
best_dfs_beta = {}
best_dfs_lr = {}
best_dfs_precond = {}
for exp in product(DATASETS, OPTIMIZERS):
    print("Finding best hyperparams for", exp)
    best_dfs_alpha[exp] = {}
    best_dfs_beta[exp] = {}
    best_dfs_lr[exp] = {}
    best_dfs_precond[exp] = {}
    exp_df = all_dfs[exp]
    # Get last metrics/performance (supposed to be epoch-smoothed for better results)
    max_ep = exp_df.groupby(ARG_COLS, sort=False)["ep"].transform(max)
    perf = exp_df[exp_df["ep"] == max_ep].drop("ep", axis=1)
    # Find the minimum aggregate metric (based on mean, median, etc.)
    def find_best_hyperparams(perf):
        if AGG == "mean":
            agg_perf = perf.groupby(ARG_COLS).mean()
        elif AGG == "median":
            agg_perf = perf.groupby(ARG_COLS).median()
        # Get the aggregated perf that minimizes the chosen metric
        min_agg_perf = agg_perf[agg_perf[METRIC] == agg_perf.min()[METRIC]]
        return min_agg_perf.index
    # Get the data associated with the args of the min aggregated metric
    exp_df = exp_df.set_index(ARG_COLS)
    best_dfs[exp] = exp_df.loc[find_best_hyperparams(perf)]
    for alpha in alphas:
        best_dfs_alpha[exp][alpha] = exp_df.loc[find_best_hyperparams(perf[perf["alpha"] == alpha])]
    for beta in betas:
        best_dfs_beta[exp][beta] = exp_df.loc[find_best_hyperparams(perf[perf["beta2"] == beta])]
    for lr in lrs:
        best_dfs_lr[exp][lr] = exp_df.loc[find_best_hyperparams(perf[perf["lr"] == lr])]
    for precond in preconds:
        best_dfs_precond[exp][precond] = exp_df.loc[find_best_hyperparams(perf[perf["precond"] == precond])]

Finding best hyperparams for ('a9a', 'Adam')
Finding best hyperparams for ('a9a', 'SARAH')
Finding best hyperparams for ('a9a', 'L-SVRG')
Finding best hyperparams for ('w8a', 'Adam')
Finding best hyperparams for ('w8a', 'SARAH')
Finding best hyperparams for ('w8a', 'L-SVRG')
Finding best hyperparams for ('rcv1', 'Adam')
Finding best hyperparams for ('rcv1', 'SARAH')
Finding best hyperparams for ('rcv1', 'L-SVRG')
Finding best hyperparams for ('real-sim', 'Adam')
Finding best hyperparams for ('real-sim', 'SARAH')
Finding best hyperparams for ('real-sim', 'L-SVRG')


In [165]:
print("Best hyperparams for each optimizer on each dataset given the following setting:")
print(FILTER_ARGS)
print()
for exp, df in best_dfs.items():
    print(exp)
    for arg, val in zip(ARG_COLS, df.index[0]):
        if arg == "lr":
            val = "2**" + str(int(log2(val)))
        print(f"- {arg} = {val}")
    print()

Best hyperparams for each optimizer on each dataset given the following setting:
{'corrupt': 'none', 'weight_decay': 0}

('a9a', 'Adam')
- lr = 2**-8
- alpha = 1e-08
- beta2 = 0.999
- precond = none

('a9a', 'SARAH')
- lr = 2**-4
- alpha = 0.001
- beta2 = 0.995
- precond = hutchinson

('a9a', 'L-SVRG')
- lr = 2**-4
- alpha = 0.001
- beta2 = 0.95
- precond = hutchinson

('w8a', 'Adam')
- lr = 2**-6
- alpha = 1e-08
- beta2 = 0.999
- precond = none

('w8a', 'SARAH')
- lr = 2**-10
- alpha = 1e-07
- beta2 = 0.999
- precond = hutchinson

('w8a', 'L-SVRG')
- lr = 2**-8
- alpha = 1e-07
- beta2 = 0.999
- precond = hutchinson

('rcv1', 'Adam')
- lr = 2**-6
- alpha = 1e-08
- beta2 = 0.995
- precond = none

('rcv1', 'SARAH')
- lr = 2**-12
- alpha = 1e-07
- beta2 = 0.95
- precond = hutchinson

('rcv1', 'L-SVRG')
- lr = 2**0
- alpha = 0.001
- beta2 = 0.95
- precond = hutchinson

('real-sim', 'Adam')
- lr = 2**-6
- alpha = 1e-08
- beta2 = 0.999
- precond = none

('real-sim', 'SARAH')
- lr = 2**-12
- 

# Plotting

## Plot gradnorm per lr

In [166]:
print("Types")
for col in df.columns:
    print(col, df[col].dtypes)

Types
ep float64
loss float64
gradnorm float64
error float64


In [167]:
print("Learning rates:")
for exp, df in all_dfs.items():
    display(set("2**"+str(int(log2(lr))) for lr in df["lr"]))
    break

Learning rates:


{'2**-10',
 '2**-12',
 '2**-14',
 '2**-16',
 '2**-2',
 '2**-4',
 '2**-6',
 '2**-8',
 '2**0',
 '2**2',
 '2**4'}

In [168]:
print("Range")
for col in df.columns:
    if df[col].dtypes != "object":
        print(f"{col}: ({df[col].min():}, {df[col].max()})")

Range
ep: (0.0, 100.0)
lr: (1.52587890625e-05, 16.0)
alpha: (1e-08, 1e-08)
beta2: (0.95, 0.999)
loss: (0.32296919432635174, 2.204534029521757)
gradnorm: (9.559924702890735e-07, 0.032853098105228115)
error: (0.15025772140495275, 0.5)


In [169]:
# Set inf values to nan and recheck range
VERYBIGNUMBER = 10**10
df[df == float("inf")] = np.nan
df[df[["loss","gradnorm"]] > VERYBIGNUMBER] = np.nan
for col in df.columns:
    if df[col].dtypes != "object":
        print(f"{col}: ({df[col].min():}, {df[col].max()})")

ep: (0.0, 100.0)
lr: (1.52587890625e-05, 16.0)
alpha: (1e-08, 1e-08)
beta2: (0.95, 0.999)
loss: (0.32296919432635174, 2.204534029521757)
gradnorm: (9.559924702890735e-07, 0.032853098105228115)
error: (0.15025772140495275, 0.5)


In [170]:
print("Check for NaNs in each column for each df.")
for col in df.columns:
    print(col)
    for exp, df in all_dfs.items():
        print(f"- {exp}: {df[col].isna().sum()}")

Check for NaNs in each column for each df.
ep
- ('a9a', 'Adam'): 0
- ('a9a', 'SARAH'): 0
- ('a9a', 'L-SVRG'): 0
- ('w8a', 'Adam'): 0
- ('w8a', 'SARAH'): 0
- ('w8a', 'L-SVRG'): 0
- ('rcv1', 'Adam'): 0
- ('rcv1', 'SARAH'): 0
- ('rcv1', 'L-SVRG'): 0
- ('real-sim', 'Adam'): 0
- ('real-sim', 'SARAH'): 0
- ('real-sim', 'L-SVRG'): 0
lr
- ('a9a', 'Adam'): 0
- ('a9a', 'SARAH'): 0
- ('a9a', 'L-SVRG'): 0
- ('w8a', 'Adam'): 0
- ('w8a', 'SARAH'): 0
- ('w8a', 'L-SVRG'): 0
- ('rcv1', 'Adam'): 0
- ('rcv1', 'SARAH'): 0
- ('rcv1', 'L-SVRG'): 0
- ('real-sim', 'Adam'): 0
- ('real-sim', 'SARAH'): 0
- ('real-sim', 'L-SVRG'): 0
alpha
- ('a9a', 'Adam'): 0
- ('a9a', 'SARAH'): 0
- ('a9a', 'L-SVRG'): 0
- ('w8a', 'Adam'): 0
- ('w8a', 'SARAH'): 0
- ('w8a', 'L-SVRG'): 0
- ('rcv1', 'Adam'): 0
- ('rcv1', 'SARAH'): 0
- ('rcv1', 'L-SVRG'): 0
- ('real-sim', 'Adam'): 0
- ('real-sim', 'SARAH'): 0
- ('real-sim', 'L-SVRG'): 0
beta2
- ('a9a', 'Adam'): 0
- ('a9a', 'SARAH'): 0
- ('a9a', 'L-SVRG'): 0
- ('w8a', 'Adam'): 0
- ('w8

In [171]:
y = "gradnorm"  # METRICS
mode = "lrs"  # (alphas, betas, lrs)

best_dfs_mode = {"alphas": best_dfs_alpha, "betas": best_dfs_beta, "lrs": best_dfs_lr}
mode_greek = {"alphas": r"$\alpha$", "betas": r"$\beta$", "lrs": r"$\eta$"}
y_greek = {"loss": r"$P(w_t)$", "gradnorm": r"$||\nabla P(w_t)||^2$", "error": "Error"}
valid_optimizers = [opt for opt in OPTIMIZERS if not (mode == "alphas" and opt == "Adam")]

start_time = time.time()
# Plot data for all optim, datasets, and args
fig, axes = plt.subplots(len(valid_optimizers), len(DATASETS))
fig.set_size_inches(5 * len(DATASETS), 5 * (len(valid_optimizers)))
title = rf"{y_greek[y]} per {mode_greek[mode]}"
plt.suptitle(title)
for i, optimizer in enumerate(valid_optimizers):
    for j, dataset in enumerate(DATASETS):
        exp = (dataset, optimizer)
        if exp not in best_dfs_lr:
            continue
        exp_df = pd.concat(list(best_dfs_mode[mode][exp].values())).reset_index()
        if len(exp_df) == 0:
            continue
        axes[i,j].set_title(rf"$\tt {optimizer}({dataset})$")
        # avoid silly problem of inconsistent style across axes
        exp_df["beta2"] = exp_df["beta2"].astype(str)
        exp_df = exp_df.sort_values("alpha", ascending=False)
        exp_df = exp_df.sort_values("beta2", ascending=True)
        print(f"Plotting lines for {exp}...")
        sns.lineplot(ax=axes[i,j], x="ep", y=y,
                     hue="lr", hue_norm=LogNorm(), palette="vlag",
                     size="beta2", style="alpha", data=exp_df)
        axes[i,j].set_ylabel(rf"{y_greek[y]}")
        axes[i,j].set(yscale="log")  # @TODO: always set log scale?
fig.tight_layout()

# Create a string out of filter args and save figure
filter_args_str = ",".join(f"{k}={v}" for k,v in FILTER_ARGS.items())
plt.savefig(f"plots/{mode}({filter_args_str}).pdf")
plt.close()
plot_per_lr_time = time.time() - start_time
print(f"Took about {plot_per_lr_time:.2f} seconds to create this plot.")

Plotting lines for ('a9a', 'Adam')...
Plotting lines for ('w8a', 'Adam')...
Plotting lines for ('rcv1', 'Adam')...
Plotting lines for ('real-sim', 'Adam')...
Plotting lines for ('a9a', 'SARAH')...
Plotting lines for ('w8a', 'SARAH')...
Plotting lines for ('rcv1', 'SARAH')...
Plotting lines for ('real-sim', 'SARAH')...
Plotting lines for ('a9a', 'L-SVRG')...
Plotting lines for ('w8a', 'L-SVRG')...
Plotting lines for ('rcv1', 'L-SVRG')...
Plotting lines for ('real-sim', 'L-SVRG')...
Took about 85.74 seconds to create this plot.


## Plot best performance of each optimizer on each dataset

In [172]:
start_time = time.time()
# Plot 3 rows each one showing some performance metric,
# where the columns are the dataset on which the optim is run.
fig, axes = plt.subplots(3, len(DATASETS))
fig.set_size_inches(5 * len(DATASETS), 5 * 3)
plt.suptitle(rf"Best Performances")
for j, dataset in enumerate(DATASETS):
    for optimizer in OPTIMIZERS:
        exp = (dataset, optimizer)
        if exp not in best_dfs:
            continue
        # Get hyperparams of best performance of 'optimizer' on 'dataset'
        args = {k:v for k,v in zip(best_dfs[exp].index.names, best_dfs[exp].index[0])}
        exp_df = best_dfs[exp].reset_index()
        # Show power of lr as 2^lr_pow
        lr_pow = round(log2(args['lr']))
        if optimizer == "Adam":
            sublabel = rf"$\eta = 2^{{{lr_pow}}}$, $\beta_1=0.9$, $\beta_2={args['beta2']}$"
        else:
            sublabel = rf"$\eta = 2^{{{lr_pow}}}$, $\alpha={args['alpha']}$, $\beta={args['beta2']}$"
        label = rf"{optimizer}({sublabel})"
        print(f"Plotting lines for {exp}...")
        sns.lineplot(x="ep", y="loss", label=label, ax=axes[0,j], data=exp_df)
        sns.lineplot(x="ep", y="gradnorm", label=label, ax=axes[1,j], data=exp_df)
        sns.lineplot(x="ep", y="error", label=label, ax=axes[2,j], data=exp_df)
    # Loss
    axes[0,j].set_title(dataset)
    axes[0,j].set_ylabel(r"$P(w_t)$")
    axes[0,j].set_xlabel("Effective Passes")
    axes[0,j].legend()
    # Gradnorm
    axes[1,j].set(yscale="log")
    axes[1,j].set_title(dataset)
    axes[1,j].set_ylabel(r"$||\nabla P(w_t)||^2$")
    axes[1,j].set_xlabel("Effective Passes")
    axes[1,j].legend()
    # Error
    axes[2,j].set(yscale="log")
    axes[2,j].set_title(dataset)
    axes[2,j].set_ylabel("Error")
    axes[2,j].set_xlabel("Effective Passes")
    axes[2,j].legend()
fig.tight_layout()

# Create a string out of filter args and save figure
filter_args_str = ",".join(f"{k}={v}" for k,v in FILTER_ARGS.items())
plt.savefig(f"plots/overall({filter_args_str}).pdf")
plt.close()
plot_best_time = time.time() - start_time
print(f"Took about {plot_best_time:.2f} seconds to create this plot.")

Plotting lines for ('a9a', 'Adam')...
Plotting lines for ('a9a', 'SARAH')...
Plotting lines for ('a9a', 'L-SVRG')...
Plotting lines for ('w8a', 'Adam')...
Plotting lines for ('w8a', 'SARAH')...
Plotting lines for ('w8a', 'L-SVRG')...
Plotting lines for ('rcv1', 'Adam')...
Plotting lines for ('rcv1', 'SARAH')...
Plotting lines for ('rcv1', 'L-SVRG')...
Plotting lines for ('real-sim', 'Adam')...
Plotting lines for ('real-sim', 'SARAH')...
Plotting lines for ('real-sim', 'L-SVRG')...
Took about 20.31 seconds to create this plot.


# Generate preconditioning comparison plots

In [ ]:
start_time = time.time()
fig, axes = plt.subplots(3, len(DATASETS))
fig.set_size_inches(5 * len(DATASETS), 5 * 3)
plt.suptitle(rf"Top performance beta")
for j, dataset in enumerate(DATASETS):
    optim_df = pd.DataFrame()
    for optimizer in OPTIMIZERS:
        exp = (dataset, optimizer)
        if exp not in best_dfs_precond:
            continue
        # Put dfs together and mark them with the optimizer's name.
        exp_df = pd.concat(list(best_dfs_precond[exp].values()))
        if len(exp_df) == 0:
            continue
        exp_df["optimizer"] = optimizer
        optim_df = optim_df.append(exp_df)
    # reset index and combine precond with gamma
    print(f"Plotting lines for {dataset}...")
    optim_df = optim_df.reset_index()
    sns.lineplot(x="ep", y="loss", hue="optimizer", style="precond", ax=axes[0,j], data=optim_df)
    sns.lineplot(x="ep", y="gradnorm", hue="optimizer", style="precond", ax=axes[1,j], data=optim_df)
    sns.lineplot(x="ep", y="error", hue="optimizer", style="precond", ax=axes[2,j], data=optim_df)
    # Loss
    axes[0,j].set_title(dataset)
    axes[0,j].set_ylabel(r"$P(w_t)$")
    axes[0,j].set_xlabel("Effective Passes")
    # Gradnorm
    axes[1,j].set(yscale="log")
    axes[1,j].set_title(dataset)
    axes[1,j].set_ylabel(r"$||\nabla P(w_t)||^2$")
    axes[1,j].set_xlabel("Effective Passes")
    # Error
    axes[2,j].set(yscale="log")
    axes[2,j].set_title(dataset)
    axes[2,j].set_ylabel("Error")
    axes[2,j].set_xlabel("Effective Passes")
fig.tight_layout()

filter_args_str = ",".join(f"{k}={v}" for k,v in FILTER_ARGS.items())
plt.savefig(f"plots/preconditioning({filter_args_str}).pdf")
plt.close()
plot_compare_time = time.time() - start_time
print(f"Took about {plot_compare_time:.2f} seconds to create this plot.")

In [ ]:
def display_best_performances(best_data, show_alpha=True):
    for dataset in DATASETS:
        for optimizer in OPTIMIZERS:
            # Extract best performance metrics for each experiment
            exp = (dataset, optimizer)
            if exp not in best_data:
                continue
            if len(best_data[exp].index) == 0:
                # Not applicable to exp, likely because it does not have 'precond'
                continue
            args = {k:v for k,v in zip(best_data[exp].index.names, best_data[exp].index[0])}
            exp_df = best_data[exp].reset_index()
            loss = exp_df["loss"].iloc[-1]
            gradnorm = exp_df["gradnorm"].iloc[-1]
            error = exp_df["error"].iloc[-1]
            # Print report
            print(f"{exp}:"
                  f"\tlr = 2^{str(round(log2(args['lr'])))}," + \
                  (f"\talpha = {args['alpha']}," if show_alpha else "") + \
                  f"\tloss = {loss:5f},"
                  f"\tgradnorm = {gradnorm:5f},"
                  f"\terror = {error:5f}")
    print()


print(f"Best hyperparameters using {METRIC} metric WITHOUT preconditoning:")
display_best_performances(best_dfs_precond["none"], show_alpha=False)
print(f"Best hyperparameters using {METRIC} metric WITH preconditoning:")
display_best_performances(best_dfs_precond["hutchinson"], show_alpha=True)